In [122]:
"""
This script will sum up utilities by month and indicate how much is to be paid by each roommate.
It should track outstanding balances too
""";

In [152]:
import pandas as pd
from openpyxl import load_workbook

import numpy as np

import os
import glob

import datetime

In [124]:
def get_dtime (string):
    """
    converts strings to datetime objects using datetime.strptime mostly just wrote this so I could .apply() it in pandas
    method chains
    
    takes (string) which is the full date of format mm/dd/yyyy (can't be 2 digit year)
    """
    return datetime.datetime.strptime(string, '%m/%d/%Y')

In [184]:
def open_hist (filepath):
    
    sheet_dict = pd.read_excel(filepath, sheet_name=None)
    
#     hist['open'] = hist['open'].apply(get_dtime)
#     hist['close'] = hist['close'].apply(get_dtime)
    
#     hist['date'] = hist['date'].apply(get_dtime)
    
#     hist['st close'] = hist['st close'].apply(get_dtime)
#     hist['st open'] = hist['st open'].apply(get_dtime)
    
    return sheet_dict

In [126]:
def arazin_bal (utils_history_tidy):
    
    #get all the utils charges that arazin hasn't paid his share yet
    ara_unpaid = utils_history_tidy.loc[utils_history_tidy['arazin paid'].astype(str) == '0', :]
    
    #when it is just arazin and I, so total is split evenly by 2
    just_2_roomies = sum(ara_unpaid.loc[ara_unpaid['alex paid'].astype(str) == 'na', 'amnt'])/2
    
    #when we have the full 3 roommates
    full_house = sum(ara_unpaid.loc[ara_unpaid['alex paid'].astype(str) != 'na', 'amnt'])/3
    
    return (just_2_roomies + full_house, ara_unpaid)

In [127]:
def alex_bal (utils_history_tidy):
    
    #get all the utils charges that alex hasn't paid his share yet
    al_unpaid = utils_history_tidy.loc[utils_history_tidy['alex paid'].astype(str) == '0', :]
    
    #when it is just alex and I, so total is split evenly by 2
    just_2_roomies = sum(al_unpaid.loc[al_unpaid['arazin paid'].astype(str) == 'na', 'amnt'])/2
    
    #when we have the full 3 roommates
    full_house = sum(al_unpaid.loc[al_unpaid['arazin paid'].astype(str) != 'na', 'amnt'])/3
    
    return (just_2_roomies + full_house, al_unpaid)

In [142]:
def bal_and_dates (utils_history_tidy, who):
    
    if who == 'arazin':
        bal, dates = arazin_bal(utils_hist)
    
    elif who == 'alex':
        bal, dates = alex_bal(utils_hist)
        
    else:
        raise NameError('This is not a valid roommate')
    
    latest_unpaid_periods = pd.DataFrame()
    
    for util in np.unique(dates['util']):
    
        idx = dates['util'] == util

        open_per_util = dates.loc[idx, 'open']

        ma = max(open_per_util)

        idx_of_max = dates['open'] == ma

        max_per = dates.loc[idx_of_max, ['open', 'close']]
        
        max_per = max_per.assign(util = util)
        
        latest_unpaid_periods = pd.concat([latest_unpaid_periods, max_per])
    
    #get the date you did the calculation for reference
    now = datetime.datetime.now()
    
    latest_unpaid_periods = latest_unpaid_periods.assign(date_calc = datetime.datetime(now.year, now.month, now.day), 
                                                         who = who,
                                                         balance = bal)
        
    return latest_unpaid_periods

In [176]:
def write_current_balances (curr_bals_for_all, utils_hist_file):
    
    #load this file as an openpyxl workbook
    book = load_workbook(utils_hist_file)
    
    #create pandas writer object with the openpyxl engine
    writer = pd.ExcelWriter(utils_hist_file, engine='openpyxl')
    
    #tell the writer object the pertinent information about the openpyxl workbook
    #so it can do operations (write) in the right places in the file
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    #because we told the excel writer object the workbook and names of the sheets
    #it can write to the correct sheet and maintain all the data without deleting worksheets etc
    curr_bals_for_all.to_excel(writer, "balances", index=False)
    
    #save it
    writer.save()
    
    return None

## Get the utilities history and outstanding balances

In [235]:
file = '../../../Finances/utils info/utilities_tidy.xlsx'

utils_dict = open_hist('../../../Finances/utils info/utilities_tidy.xlsx')

utils_hist = utils_dict['utils']
balances = utils_dict['balances']

In [236]:
balances

,open,close,util,date_calc,who,balance
0,2018-09-30,2018-10-30,elec,2018-11-18,arazin,155.84
1,2018-09-10,2018-10-11,gas,2018-11-18,arazin,155.84
2,2018-11-15,2018-12-14,internet,2018-11-18,arazin,155.84


### get the new balances due

In [195]:
all_new_bals = pd.DataFrame()
for roomie in ['arazin', 'alex']:
    
    new_balance = bal_and_dates(utils_hist, roomie)
    
    all_new_bals = pd.concat([all_new_bals, new_balance], axis='index', sort=False)
    
balances = pd.concat([balances, all_new_bals], axis=0)

,open,close,util,date_calc,who,balance
0,2018-09-30,2018-10-30,elec,2018-11-18,arazin,155.84
1,2018-09-10,2018-10-11,gas,2018-11-18,arazin,155.84
2,2018-11-15,2018-12-14,internet,2018-11-18,arazin,155.84
187,2018-09-30,2018-10-30,elec,2018-11-18,arazin,155.84
185,2018-09-10,2018-10-11,gas,2018-11-18,arazin,155.84
189,2018-11-15,2018-12-14,internet,2018-11-18,arazin,155.84


### update the ledger with the new balances due

In [196]:
#this does not currently check if I'm duplicating a balance
write_current_balances(balances, file)

## Deal with a payment

In [255]:
amount_paid = 155.84

the method:

-the balance sheet associates the latest unpaid bill for each utility with a balance amount

-when you search for the amount paid (assuming its exactly the amount owed), you get the latest unpaid open/close period for each utility, associated with THAT particular balance amount. (this allows me to update the utilities history and then get paid for a previous utilities amount and still update correctly)

-for each utility, find that latest unpaid open/close date in the utilities history

-update that bill open/close period and all previous bill periods (including mostly paid, but some unpaid one) to 'paid'

In [256]:
#get all the balance sheet entries for the amount paid
#(should be a single instance of each utility with open/close dates,
#these dates are the latest unpaid period for that util at that balance amount)

entries_for_amount = balances.loc[balances['balance'] == amount_paid, ['open', 'close', 'util']]

In [257]:
entries_for_amount

,open,close,util
0,2018-09-30,2018-10-30,elec
1,2018-09-10,2018-10-11,gas
2,2018-11-15,2018-12-14,internet


In [262]:
#for each utility associated with the paid balance
for util in np.unique(entries_for_amount['util']):

    #get the latest entry for a particular utility for the amount paid
    paid_entry = entries_for_amount.loc[entries_for_amount['util'] == util, :]
    
    #find which entry, and all previous, in the utilities history is being paid in this amount
    open_idx = utils_hist['open'] <= paid_entry['open'].values[0]
    close_idx = utils_hist['close'] <= paid_entry['close'].values[0]
    util_idx = utils_hist['util'] == paid_entry['util'].values[0]


    #update this and all previous entries for this utility to 'paid' for this roommate
    utils_hist.loc[open_idx & close_idx & util_idx, 'arazin paid'] = 1

## this works to update all 0's (unpaid) entries equal to or earlier than the dates of the most recent balance to 1's (paid), but will overwrite 'na' to 1, which should be avoided.

## also still need to update the balance sheet and delete the entries associated with a particular payment amount

In [263]:
utils_hist

,open,close,util,amnt,note,arazin paid,alex paid
0,2013-08-01,2013-08-14,gas,28.50,includes opening fees,1,1
1,2013-08-01,2013-08-28,elec,87.00,NaN,1,1
2,2013-08-15,2013-09-04,internet,60.00,includes opening fees,1,1
3,2013-08-14,2013-09-13,gas,6.00,NaN,1,1
4,2013-08-29,2013-09-13,elec,108.00,NaN,1,1
5,2013-09-13,2013-10-12,gas,6.00,NaN,1,1
6,2013-09-15,2013-10-14,internet,30.00,NaN,1,1
7,2013-09-26,2013-10-28,elec,33.00,NaN,1,1
8,2013-10-13,2013-11-12,gas,9.00,NaN,1,1
9,2013-10-15,2013-11-14,internet,30.00,NaN,1,1
